In [1]:
using Apophis, BenchmarkTools
using Symbolics: build_function, jacobian, derivative, Num, @variables
using LinearAlgebra
using SparseArrays: sparse, SparseMatrixCSC
using DifferentialEquations: ODEProblem, solve
using Sundials: CVODE_BDF

In [62]:
init(:GRI3, 1000.0, CH4 = 0.05, N2 = 0.75, O2 = 0.20)
step!(gas, gas.initial.mass_fractions, gas.initial.temperature)

In [4]:
gas.intermediate.production_rate

53-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [16]:
struct Buff3{T}
    # dsds::Tuple{Vector{T},Vector{T}}
    # dxdx::Tuple{Diagonal{T,Vector{T}},Diagonal{T,Vector{T}}}
    # dxdx::Tuple{Diagonal{T,Vector{T}},Diagonal{T,Vector{T}}}
    matrix_vector_multiplication::Tuple{Matrix{T}, Matrix{T}}
end

In [20]:
const buff5 = Buff3((zeros(53, 325), zeros(53, 325)))

Buff3{Float64}(([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]))

In [95]:
function rule2(::typeof(*), A, x, dz) # z = A * x
    
    nA = nothing # size(A, 1)
    dA = nothing # @thunk dz * kron(x', I(nA))
    dx = dz * A
    return dA, dx
end

rule2 (generic function with 2 methods)

In [67]:
dz = I(53)
A = Array(gas.mechanism.stoichiometric_transpose)
x = rand(1:5, 325)

325-element Vector{Int64}:
 1
 1
 1
 2
 3
 4
 5
 3
 2
 5
 ⋮
 3
 1
 1
 2
 3
 1
 1
 2
 3

In [88]:
c = A*x
function mult!(c, A, x)
    for (j, i) in gas.mechanism.reactants_indices
        c[i, 1] += A[i, j] * x[j, 1]
    end
end

mult! (generic function with 1 method)

In [89]:
@btime mult!(c, A, x)
@btime A * x
@btime mul!(c, A, x)

ErrorException: iteration is deliberately unsupported for CartesianIndex. Use `I` rather than `I...`, or use `Tuple(I)...`

In [37]:
const buff2 = Buff2((a, b), (Diagonal(a), Diagonal(b)))

Buff2{Int64}(([3, 1, 3], [1, 3, 1]), ([3 0 0; 0 1 0; 0 0 3], [1 0 0; 0 3 0; 0 0 1]))

In [12]:
e(x::AbstractVector{T}) where {T<:Real} = exp.(x)
lg(x::AbstractVector{T}) where {T<:Real} = log10.(x)
ln(x::T) where {T<:Real} = log(x)
Π(A::AbstractMatrix{T}) where {T<:Real} = prod(A, dims=2)

function rrule(::typeof(ln), x::T, Δz) where {T<:Real}

    dx = Δz * inv(x)
    return NoTangent(), dx
end

function rrule(::typeof(^), x::T, y::T, Δz) where {T<:Real}

    dx = Δz * y * x^(y - 1)
    dy = Δz * x^y * log(x)
    return NoTangent(), dx, dy
end


function rrule(::typeof(/), x::T, y::T, Δz) where {T<:Real}

    dx = Δz * inv(y)
    dy = Δz * -x * inv(y)^2
    return NoTangent(), dx, dy
end

function rrule(::typeof(⋅), x::AbstractVector{T}, y::AbstractVector{T}, Δz) where {T<:Real}

    z = x ⋅ y
    dx = y' * Δz
    dy = x' * Δz
    return NoTangent(), dx, dy
end

function rrule(::typeof(e), x::AbstractVector{T}, Δz) where {T<:Real}
    z = e(x)
    dx = Δz * Diagonal(z)
    return NoTangent(), dx
end

function rrule(::typeof(lg), x::AbstractVector{T}, Δz) where {T<:Real}
    z = lg(x)

    #d = log(10.0) * x
    #map!(x -> iszero(x) ? zero(x) : inv(x), d, d)
    #dx = Δz * Diagonal(d)
    dx = Δz / Diagonal(log(10) .* x)
    return NoTangent(), dx

end

function rrule(::typeof(.+), x::AbstractVector{T}, y::T, Δz) where {T<:Real}
    z = x .+ y
    nx = length(x)
    dx = Δz * I(nx)
    dy = Δz * ones(nx) * sign(y)
    return NoTangent(), dx, dy
end

function rrule(::typeof(.*), x::AbstractVector{T}, y::AbstractVector{T}, Δz) where {T<:Real}
    z = x .* y
    dx = Δz * Diagonal(y)
    dy = Δz * Diagonal(x)
    return NoTangent(), dx, dy
end

function rrule(::typeof(*), x::T, y::AbstractVector{T}, Δz) where {T<:Real}
    z = x * y
    ny = length(y)
    dx = Δz * y
    dy = Δz * (x)I(ny)
    return NoTangent(), dx, dy
end

function rrule(::typeof(*), A::AbstractMatrix, x::AbstractVector, Δz)
    z = A * x
    nA = size(A, 1)
    dA = @thunk Δz * kron(x', I(nA))
    dx = Δz * A
    return NoTangent(), dA, dx
end

function rrule(::typeof(*), x::AbstractVector{T}, y::T, Δz) where {T<:Real}
    z = x * y
    nx = length(x)
    dx = Δz * (y)I(nx)
    dy = Δz * x
    return NoTangent(), dx, dy
end

function rrule(::typeof(*), x::T, y::T, Δz) where {T<:Real}
    z = x * y
    dx = Δz * y
    dy = Δz * x
    return NoTangent(), dx, dy
end

function rrule(::typeof(./), x::AbstractVector{T}, y::AbstractVector{T}, Δz) where {T<:Real}
    z = x ./ y
    #y⁻¹ = similar(y)
    #nxy⁻¹ = similar(y)
    #map!(x -> iszero(x) ? zero(x) : inv(x), y⁻¹, y)
    #map!((a, b) -> -a * b, nxy⁻¹, z, y⁻¹)
    dx = Δz * Diagonal(inv.(y))
    dy = Δz * Diagonal(-x ./ y .^ 2)
    #dx = Δz * Diagonal(y⁻¹); dy = Δz * Diagonal(nxy⁻¹)
    return NoTangent(), dx, dy
end

function rrule(::typeof(/), x::AbstractVector{T}, y::T, Δz) where {T<:Real}
    z = x / y
    #nx = length(x)
    #y⁻¹ = inv(y)
    #xy⁻¹ = z * y⁻¹
    #map!(x -> -x, xy⁻¹, xy⁻¹)
    dx = Δz * I(length(x)) * inv(y)
    dy = Δz * (-x ./ y .^ 2)
    #dx = Δz * (y⁻¹)I(nx); dy = Δz * xy⁻¹
    return NoTangent(), dx, dy
end

function rrule(::typeof(./), x::T, y::AbstractVector{T}, Δz) where {T<:Real}
    z = x ./ y
    #y⁻¹ = similar(y)
    #nxy⁻¹ = similar(y)
    #map!(x -> iszero(x) ? zero(x) : inv(x), y⁻¹, y)
    #map!((a, b) -> -a * b, nxy⁻¹, z, y⁻¹)
    invY = inv.(y)
    invY[isinf.(invY)] .= 0
    dx = Δz * invY
    dy = Δz * Diagonal(-x ./ y .^ 2)
    #dx = Δz * y⁻¹; dy = Δz * Diagonal(nxy⁻¹)
    return NoTangent(), dx, dy
end

function rrule(::typeof(.^), x::T, y::AbstractVector{T}, Δz) where {T<:Real}

    #z = x .^ y
    #Δ = similar(y)
    #δ = z * log(x)
    #map!((a, b) -> a * b * inv(x), Δ, z, y)
    #
    #dx = @thunk Δz * Δ
    #dy = Δz * Diagonal(δ)
    dx = Δz * (y .* x .^ (y .- 1.0))
    dy = Δz * Diagonal(x .^ y .* log.(x))
    return NoTangent(), dx, dy
end

function rrule(::typeof(.^), x::AbstractVector{T}, y::T, Δz) where {T<:Real}

    #z = x .^ y
    #Δ = similar(x)
    #δ = similar(x)
    #map!((a, b) -> iszero(b) ? zero(b) : y * a * inv(b), Δ, z, x)
    #map!((a, b) -> iszero(b) ? zero(b) : a * log(abs(b)), δ, z, x)
    #
    #dx = Δz * Diagonal(Δ)
    #dy = @thunk Δz * δ
    dx = Δz * Diagonal(y .* x .^ (y .- 1.0))
    return NoTangent(), dx, nothing
end

#function rrule(::typeof(.^), x, A::AbstractMatrix{T}, Δz) where T<:Real
#    
#    z = x .^ A
#        
#    ni = size(A, 1); nj = size(A, 2)
#    Δx = spzeros(ni * nj, nj)
#    ΔA = (1.0)I(ni * nj)
#    k = 0
#
#    for j in 1:nj, i in 1:ni
#        k += 1
#        iszero(x[j]) ? Δx[i + ni * (j - 1), j] = zero(x[j]) : 
#        Δx[i + ni * (j - 1), j] = A[i, j] * z[i, j] * inv(x[j]) 
#        iszero(x[j]) ? ΔA[k, k] = zero(x[j]) : ΔA[k, k] = z[i, j] * log(abs(x[j]))
#    end
#
#    dx = Δz * Δx
#    dA = @thunk Δz * ΔA
#
#    return NoTangent(), dx, dA
#end

function rrule(::typeof(.^), x, y::AbstractMatrix{T}, z̄) where {T<:Real}
    #vectorized power 
    z = x .^ y

    x̄ = y .* x .^ (y .- 1.0)
    ȳ = z .* log.(abs.(x))
    x̄[isnan.(x̄)] .= 0
    ȳ[isnan.(ȳ)] .= 0
    ȳ[isinf.(ȳ)] .= 0

    m = size(y, 1)
    n = size(y, 2)
    dx = spzeros(n * m, n)
    dy = (1.0)I(n * m)

    for j in 1:n
        for i in 1:m
            dx[i+m*(j-1), j] = x̄[i, j]
        end
    end

    for i in 1:n*m
        dy[i, i] = ȳ[i]
    end

    xbar = z̄ * dx
    ybar = z̄ * dy

    return NoTangent(), xbar, ybar
end

#function rrule(::typeof(Π), A::AbstractMatrix{T}, Δz) where T<:Real
#
#    z = Π(A)
#        
#    prod = one(T)
#    ni = size(A, 1); nj = size(A, 2)
#    ΔA = zeros(ni, ni * nj)
#
#    for i in 1:ni, j in 1:nj
#            prod = one(T)
#            for k in 1:nj
#                j == k ? continue : prod *= A[i, k]
#            end
#            ΔA[i, (j - 1)ni + i] = prod
#    end
#    dA = Δz * ΔA
#    return NoTangent(), dA
#end

function rrule(::typeof(Π), A::AbstractMatrix{T}, Δz) where {T<:Real}
    #vectorized power 
    z = Π(A)


    x̄ = similar(A)
    for i in 1:size(A, 2)
        @views prod!(x̄[:, i], A[:, 1:end.!=i])
    end
    #x̄[isnan.(x̄)] .= 0

    n = size(A, 1)
    m = size(A, 2)
    M = zeros(n, n * m)

    for i in 1:n
        for j in 1:m
            M[i, (j-1)n+i] = x̄[(j-1)n+i]
        end
    end

    Ā = Δz * M
    return NoTangent(), Ā
end

LoadError: LoadError: UndefVarError: @thunk not defined
in expression starting at /Users/sabry/Developer/Apophis.jl/rules.ipynb:78

In [13]:
function rule(::typeof(⋅), x::Vector{T}, y::Vector{T}, dz::T) where {T<:Real}

    dx = first(buffs.dsds)
    dy = last(buffs.dsds)

    for i in eachindex(x)
        dx[i] = y[i] * dz
        dy[i] = x[i] * dz
    end

    return nothing
end

rule (generic function with 1 method)

In [84]:
e(x::AbstractVector{T}) where {T<:Real} = exp.(x)

function rule(::typeof(e), x::Vector{T}, dz::Matrix{T}) where {T<:Real}

    dx = first(buff2.dxdx)
    for i in eachindex(x)
        d = 4i - 3
        dx[d] = dz[d] * x[i]
    end
    return nothing
end

rule (generic function with 4 methods)

In [ ]:
function rrule(::typeof(.*), x::Vector{T}, y::Vector{T}, Δz::Matrix{T}) where {T<:Real}
    z = x .* y
    dx = Δz * Diagonal(y)
    dy = Δz * Diagonal(x)
    return NoTangent(), dx, dy
end

In [85]:
a = rand(1:3, 3)
b = rand(1:3, 3)

3-element Vector{Int64}:
 2
 3
 3

In [86]:
dz = [1 2 3; 4 5 6; 7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [87]:
rule(e, a, dz)

In [88]:
a

3-element Vector{Int64}:
 3
 2
 3

In [89]:
first(buff2.dxdx)

3×3 Diagonal{Int64, Vector{Int64}}:
 3   ⋅   ⋅
 ⋅  10   ⋅
 ⋅   ⋅  27

In [ ]:
function rates((; current, intermediate, mechanism)::Gas)

    nt = length(mechanism.threebody_reactions)
    ne = length(mechanism.elementary_reactions)
    threebody_range = ne+1:ne+nt

    W⁻¹ = mechanism.inverse_molecular_weight
    Y = current.mass_fractions
    X = current.molar_fractions
    reactantsindices = mechanism.reactants_indices
    productsindices = mechanism.products_indices
    C = current.molar_concentration
    W = mechanism.molecular_weight

    nur = mechanism.stoichiometric_reactants
    nup = mechanism.stoichiometric_products
    nut = mechanism.stoichiometric_transpose

    cᵥ = intermediate.heat_capacity_volume
    u = intermediate.internal_energy
    kf = intermediate.forward_rate_constant
    kr = intermediate.reverse_rate_constant
    M = view(intermediate.total_molar_concentration, 1:nt)
    q = intermediate.rate_of_progress
    ω̇ = intermediate.production_rate
    Ẏ = intermediate.mass_change_rate
    Ṫ = intermediate.temperature_change_rate

    ρ = current.density

    @inbounds for ci in reactantsindices ## combine into one function
        i, j = Tuple(ci)
        kf[i] *= C[j]^nur[ci]
    end

    @inbounds for ci in productsindices
        i, j = Tuple(ci)
        kr[i] *= C[j]^nup[ci]
    end

    for k in eachindex(q)
        q[k] = kf[k] - kr[k]
        k ∈ threebody_range && (q[k] *= M[k-ne])
    end

    mul!(ω̇, nut, q)

    W̅ = inv(Y ⋅ W⁻¹)
    c̅ᵥ = cᵥ ⋅ X / W̅

    for i in eachindex(Ẏ)
        Ẏ[i] = ω̇[i] * W[i] / ρ
    end

    Ṫ[1] = -(ω̇ ⋅ u) / (ρ * c̅ᵥ) ## worth it?

    return nothing
end